# Import Libraries

In [30]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from sklearn.metrics import confusion_matrix

plt.ion()   # interactive mode

# Utils

#### Visualize model

In [31]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

#### imshow

In [32]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# # Get a batch of training data
# inputs, classes = next(iter(dataloaders['train']))

# # Make a grid from batch
# out = torchvision.utils.make_grid(inputs)

# imshow(out, title=[class_names[x] for x in classes])

#### Train model

In [38]:
#https://scikit-learn.org/dev/auto_examples/model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-selection-plot-confusion-matrix-py
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


np.set_printoptions(precision=2)

In [39]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_labels = []
    best_preds = []

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            running_labels = []
            running_preds = []

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                running_labels.extend(list(labels.data.cpu()))
                running_preds.extend(list(preds.cpu()))

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                best_labels = running_labels.copy()
                best_preds = running_preds.copy()

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    
    print('Confusion matrix')
    plot_confusion_matrix(np.array(best_labels, dtype=np.int32), 
                          np.array(best_preds, dtype=np.int32), class_names)

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

#### Load data

In [40]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation((-45, 45)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(), 
        transforms.ToTensor(),
        transforms.ColorJitter(), 
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'dataset'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=64,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#### Experiment 1: no freezing

In [41]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, len(class_names))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=1e-2, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------


Exception ignored in: <function _DataLoaderIter.__del__ at 0x7fe58f9850d0>
Exception ignored in: <function _DataLoaderIter.__del__ at 0x7fe58f9850d0>
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 717, in __del__
Exception ignored in: <function _DataLoaderIter.__del__ at 0x7fe58f9850d0>
    self._shutdown_workers()
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 717, in __del__
Traceback (most recent call last):
Exception ignored in: <function _DataLoaderIter.__del__ at 0x7fe58f9850d0>
    self._shutdown_workers()
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 713, in _shutdown_workers
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 717, in __del__
Traceback (most recent call last):
    w.join()
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/ut

train Loss: 0.9438 Acc: 0.7962
val Loss: 0.9913 Acc: 0.6052

Epoch 1/24
----------
train Loss: 0.5997 Acc: 0.8519
val Loss: 0.6734 Acc: 0.7296

Epoch 2/24
----------
train Loss: 0.9254 Acc: 0.7635
val Loss: 0.6481 Acc: 0.8798

Epoch 3/24
----------
train Loss: 0.4748 Acc: 0.8596
val Loss: 0.4300 Acc: 0.8112

Epoch 4/24
----------
train Loss: 0.3935 Acc: 0.8750
val Loss: 0.3578 Acc: 0.8841

Epoch 5/24
----------
train Loss: 0.3834 Acc: 0.8885
val Loss: 0.4240 Acc: 0.8326

Epoch 6/24
----------
train Loss: 0.3252 Acc: 0.8865
val Loss: 0.3723 Acc: 0.8841

Epoch 7/24
----------
train Loss: 0.2780 Acc: 0.8981
val Loss: 0.3643 Acc: 0.8798

Epoch 8/24
----------
train Loss: 0.2970 Acc: 0.8846
val Loss: 0.3535 Acc: 0.8841

Epoch 9/24
----------
train Loss: 0.2711 Acc: 0.9000
val Loss: 0.3349 Acc: 0.8798

Epoch 10/24
----------
train Loss: 0.2917 Acc: 0.8885
val Loss: 0.3197 Acc: 0.8798

Epoch 11/24
----------
train Loss: 0.2655 Acc: 0.8865
val Loss: 0.3173 Acc: 0.8798

Epoch 12/24
----------
t

In [ ]:
visualize_model(model_ft)
plt.show()

##### Experiment 2: freezing except last layer

In [11]:
model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, len(class_names))

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [12]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 0.2624 Acc: 0.9038
val Loss: 0.1792 Acc: 0.9356

Epoch 1/24
----------
train Loss: 0.2493 Acc: 0.9154
val Loss: 0.1750 Acc: 0.9399

Epoch 2/24
----------
train Loss: 0.2498 Acc: 0.8981
val Loss: 0.1751 Acc: 0.9442

Epoch 3/24
----------
train Loss: 0.2754 Acc: 0.9115
val Loss: 0.1734 Acc: 0.9399

Epoch 4/24
----------
train Loss: 0.2668 Acc: 0.9154
val Loss: 0.1786 Acc: 0.9399

Epoch 5/24
----------
train Loss: 0.2591 Acc: 0.9077
val Loss: 0.1826 Acc: 0.9270

Epoch 6/24
----------
train Loss: 0.2880 Acc: 0.9019
val Loss: 0.1836 Acc: 0.9356

Epoch 7/24
----------
train Loss: 0.2617 Acc: 0.9096
val Loss: 0.1802 Acc: 0.9356

Epoch 8/24
----------
train Loss: 0.2820 Acc: 0.9135
val Loss: 0.1920 Acc: 0.9313

Epoch 9/24
----------
train Loss: 0.2785 Acc: 0.9038
val Loss: 0.1839 Acc: 0.9356

Epoch 10/24
----------
train Loss: 0.3011 Acc: 0.9096
val Loss: 0.1876 Acc: 0.9356

Epoch 11/24
----------
train Loss: 0.3030 Acc: 0.9019
val Loss: 0.1795 Acc: 0.9356

Ep